1. Create an S3 Bucket to upload bike data CSV file

In [22]:
!aws s3api create-bucket --bucket aws-glue-bike-data-activity-file-upload --region us-east-2 --create-bucket-configuration LocationConstraint=us-east-2 

{
    "Location": "http://aws-glue-bike-data-activity-file-upload.s3.amazonaws.com/"
}


2. Copy initial CSV file into S3 Bucket

In [24]:
!aws s3 cp Activities.csv s3://aws-glue-bike-data-activity-file-upload/ 

upload: ./Activities.csv to s3://aws-glue-bike-data-activity-file-upload/Activities.csv


3. Run ETL Job on Uploaded File

In [2]:
from pyspark.sql import SparkSession 
from awsglue.context import GlueContext 

# Create a SparkSession 
spark = SparkSession.builder.appName("Read from S3").getOrCreate() 

# Create a AWS Glue Context
glueContext = GlueContext(spark.sparkContext) 

# S3 bucket and prefix (folder) containing CSV files
s3_bucket = "bike-data-activity-file-upload"

# Construct the S3 path pattern
s3_path_pattern = f"s3://bike-data-activity-file-upload/Activities.csv"

# Read all CSV files from S3 into a single DataFrame
df = spark.read.csv(s3_path_pattern, header=True, inferSchema=True)

record_count = df.count()

# Display the number of records
print("Number of records:", record_count)

# Drop duplicate records from CSVs
df_dropped_duplicates = df.dropDuplicates();
record_count = df_dropped_duplicates.count()
print("Number of records ():", record_count)

# Filter out activites that are not Road Cycling 
filtered_data = df_dropped_duplicates.filter(df["Activity Type"] == "Road Cycling")

# Drop columns we are not interested in 
columns_to_drop = ["Activity Type", "Favorite", "Avg Vertical Ratio", "Avg Vertical Oscillation", "Avg Ground Contact Time", "Training Stress Score®", "Grit", "Flow", "Surface Interval", "Decompression", "Best Lap Time", "Number of Laps", "Avg. Swolf", "Dive Time", "Avg Stride Length", "Aerobic TE", "Total Reps", "Min Temp", "Max Temp", "Total Strokes" , "Avg Stroke Rate", "Avg Resp", "Min Resp", "Max Resp", "Moving Time"]
transformed_data = filtered_data.drop(*columns_to_drop)
transformed_data.show(5)

# Define the output S3 path (destination bucket)
output_path = f"s3://{s3_bucket}/processed-data/"

# Write the processed data to the destination S3 bucket as a CSV file
transformed_data.coalesce(1).write.option("header", "true").mode("overwrite").csv(output_path)



Number of records: 20
Number of records (): 20
+-------------------+--------------------+--------+--------+----------+------+------+---------+---------+------------+-------------+----------------+----------------+---------+---------+------------+-------------+-------------+
|               Date|               Title|Distance|Calories|      Time|Avg HR|Max HR|Avg Speed|Max Speed|Total Ascent|Total Descent|Avg Bike Cadence|Max Bike Cadence|Avg Power|Max Power|Elapsed Time|Min Elevation|Max Elevation|
+-------------------+--------------------+--------+--------+----------+------+------+---------+---------+------------+-------------+----------------+----------------+---------+---------+------------+-------------+-------------+
|2024-01-18 14:57:34|Folly Beach Road ...|    0.05|       2|00:00:21.8|    98|   100|      8.1|     10.1|          --|           --|              52|              58|        0|        0|  00:00:21.8|           52|           54|
|2024-02-20 12:50:51|Chicago Road Cycling

4. Create job in AWS Glue for automtion